# Multi-Year Zonal Mode Aggregation for Crop Data Layers

Author: Shuo Yu

Date: 2022/08

---

## Description
This script computes zonal statistics (mode values) for raster crop classification layers (CDL) over multiple years using field boundary polygons (CLU shapefile).

---

## Workflow
1. Load shapefile of field boundaries (with unique ID).  
2. Load matching CSV with the same IDs for later merging.  
3. For each CDL raster (per year):  
   - Reproject field boundaries to match raster CRS.  
   - Calculate the mode of CDL categories within each polygon.  
   - Append the result as a new column (e.g., `CDL2013`).  
4. Merge all results back with the input CSV on `ID`.  
5. Save final merged dataset to CSV.  

---

## Notes
- CRS must match between raster and vector. Here we use **EPSG:26915**.  
- CDL background values (`0`) are treated as nodata and ignored.  
- The mode is computed using categorical zonal counts.  
- This version avoids per-geometry loops for efficiency.  

---

## Output
- DataFrame with original CSV columns plus CDL values for each year.  
- Shapefile of merged DataFrame.  

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
import time
import numpy as np
import pandas as pd
import osgeo
import geopandas as gpd
from shapely.geometry import MultiLineString, LineString, Point, MultiPolygon, Polygon, MultiPoint
from shapely.ops import split, substring
from rasterio import features
import rasterio
import rasterio.transform
import rasterio.mask
import rasterio.warp
import rasterio.windows
from rasterio.transform import from_origin
from rasterio.features import rasterize
from shapely.geometry import shape
import fiona
from pyproj import CRS, Transformer

In [ ]:
# Parameters
threshold_acreage = 1
neg_buffer = 30
cwd = os.getcwd()
state_abb = 'IL'
state_name = 'Illinois'
state_fips = '26'
CLU_directory = r'D:\CLU\mi'
CSB1623_address = r'D:\School\Research\CoverCropDetection\Data_Original\NationalCSB_2016-2023_rev23\CSB1623.gdb'
CSB1320_address = r'D:\School\Research\CoverCropDetection\Data_Original\2020_National_CSB_gdb\National_Final_gdb\CSB1320.gdb'
# Shapefiles of NationalCSB
# Path to the .gdb file
gdb_path = CSB1623_address
county_shapefile_address = r'E:\CoverCrop\Data_Original\cb_2018_us_county_20m.zip'
output_dir = os.path.join(cwd, rf'Data_cleaned\CLU_{state_abb}')
# Create the directory, existing = ok
os.makedirs(output_dir, exist_ok=True)

### 1. Read in CSB data

In [ ]:
# Updated dictionary for mapping STATEFP to state names
statefp_to_state_name = {
    '17': 'Illinois',
    '18': 'Indiana',
    '19': 'Iowa',
    '26': 'Michigan',
    '27': 'Minnesota',
    '31': 'Nebraska',
    '39': 'Ohio',
    '46': 'South Dakota',
    '55': 'Wisconsin',
    '29': 'Missouri',
    '05': 'Arkansas'
}

In [ ]:
# Read the shapefile into a GeoDataFrame
counties = gpd.read_file(county_shapefile_address)
# Dissolve counties to create state boundaries
states = counties.dissolve(by='STATEFP').reset_index()
states.to_crs('EPSG:5070', inplace=True)

In [ ]:
# List all available layers in the .gdb file
layer_name = fiona.listlayers(gdb_path)[0]

In [ ]:
# List all available layers and their CRS information
try:
    with fiona.Env():
        for layer in fiona.listlayers(gdb_path):
            with fiona.open(gdb_path, layer=layer) as src:
                print(f"Layer: {layer}")
                print(f"CRS: {src.crs}")
                print(f"CRS WKT: {src.crs_wkt}")
                print('-' * 40)
except Exception as e:
    print(f"Error accessing .gdb file: {e}")

In [ ]:
# Function to create a dictionary of bounds
def get_bounds_dict(geometry):
    minx, miny, maxx, maxy = geometry.bounds
    return {
        'xmax': maxx+100,
        'xmin': minx-100,
        'ymax': maxy+100,
        'ymin': miny-100
    }

# Create a dictionary of dictionaries
bounds_dict = {row['STATEFP']: 
               get_bounds_dict(row.geometry) 
               for idx, row in states.iterrows()}

In [ ]:
def change_crs(minx, miny, maxx, maxy, input_crs):
    # Create Transformer objects
    transformer = Transformer.from_crs(input_crs, src.crs, always_xy=True)

    # Transform the bounding box to Albers Equal Area coordinates
    xmin_albers, ymin_albers = transformer.transform(minx, miny)
    xmax_albers, ymax_albers = transformer.transform(maxx, maxy)

    # Define the projected bounding box
    projected_bbox = (xmin_albers, ymin_albers, xmax_albers, ymax_albers)
    return projected_bbox

### 2. Transfer CSB to Raster Data

In [ ]:
geographic_bbox = bounds_dict[state_fips]
projected_bbox = change_crs(geographic_bbox['xmin'], geographic_bbox['ymin'],
                            geographic_bbox['xmax'], geographic_bbox['ymax'],
                            states.crs)
# Read the layer with the projected bounding box filter
CSB = gpd.read_file(gdb_path, layer=layer_name, bbox=projected_bbox)
CSB = CSB.to_crs('EPSG:26915')
CSB = CSB.drop(columns=['CSBID', 'CSBYEARS', 'CSBACRES', 'STATEFIPS', 'STATEASD', 
                              'ASD', 'CNTY', 'CNTYFIPS', 'INSIDE_X', 'INSIDE_Y',
                              'Shape_Length', 'Shape_Area'])
CSB.to_file(os.path.join(output_dir, f'CSB_{state_abb}.shp'))

In [ ]:
# Define the output raster properties
output_crs = CSB.crs  # Use the same CRS as the GeoDataFrame
resolution = 30  # Define the desired resolution in map units (e.g., meters)
cols = CSB.columns.drop('geometry')  # Columns to rasterize

# Calculate the bounds of the raster from the bounds of the GeoDataFrame
minx, miny, maxx, maxy = CSB.total_bounds
width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)

# Define the transform for the raster (affine transformation matrix)
transform = from_origin(minx, maxy, resolution, resolution)

# Function to rasterize a single column
def rasterize_column(gdf, column_name):
    shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf[column_name]))
    raster = rasterize(
        shapes,
        out_shape=(height, width),
        transform=transform,
        fill=np.nan,  # Value to use for areas outside of the geometries
        dtype='float32'  # Change dtype as needed
    )
    return raster

for column in cols:
    raster_data = rasterize_column(CSB, column)
    
    # Define the output filename and path
    output_filename = f"{state_abb}_{column}.tif"
    output_path = os.path.join(output_dir, output_filename)
    
    # Write the raster to a file
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,  # Number of bands
        dtype=raster_data.dtype,
        crs=output_crs,
        transform=transform,
    ) as dst:
        dst.write(raster_data, 1)

In [ ]:
# Define the output raster properties
output_crs = CSB.crs  # Use the same CRS as the GeoDataFrame
resolution = 30  # Define the desired resolution in map units (e.g., meters)
cols = CSB.columns.drop('geometry')  # Columns to rasterize

# Calculate the bounds of the raster from the bounds of the GeoDataFrame
minx, miny, maxx, maxy = CSB.total_bounds
width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)

# Define the transform for the raster (affine transformation matrix)
transform = from_origin(minx, maxy, resolution, resolution)

# Function to rasterize a single column
def rasterize_column(gdf, column_name):
    shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf[column_name]))
    raster = rasterize(
        shapes,
        out_shape=(height, width),
        transform=transform,
        fill=np.nan,  # Value to use for areas outside of the geometries
        dtype='float32'  # Change dtype as needed
    )
    return raster

for column in cols:
    raster_data = rasterize_column(CSB, column)
    
    # Define the output filename and path
    output_filename = f"{state_abb}_{column}.tif"
    output_path = os.path.join(output_dir, output_filename)
    
    # Write the raster to a file
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,  # Number of bands
        dtype=raster_data.dtype,
        crs=output_crs,
        transform=transform,
    ) as dst:
        dst.write(raster_data, 1)

### 3. Read in CLU Data

In [ ]:
# Function to generate a random point within a polygon
def random_point_in_polygon(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    while True:
        p = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
        if polygon.contains(p):
            return p

# Function to extract value at points in gdf_points from a raster
def extract_raster_values(gdf_points, year):
    # Path to the GeoTIFF file
    raster_path = os.path.join(rf'Data_cleaned\CLU_{state_abb}', f'{state_abb}_CDL{year}.tif')
    
    # Extract raster values at each point
    raster_values = []
    
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs  # Get the CRS of the raster
        # Check if the CRS of the points matches the raster CRS
        if gdf_points.crs != raster_crs:
            # Reproject the points to the raster CRS if they do not match
            gdf_points = gdf_points.to_crs(raster_crs)
        
        for point in gdf_points.geometry:
            if point is None:
                raster_values.append(None)
            else:
                # Get the raster value at the point
                coords = [(point.x, point.y)]
                value = list(src.sample(coords))[0][0]
                raster_values.append(value)
    
    # Add the raster values to the points GeoDataFrame
    gdf_points[f'CDL{year}'] = raster_values
    return gdf_points

In [ ]:
# Start timing
start_time = time.time()

# List to hold processed GeoDataFrames
list_gdfs = []

# Iterate through each folder in the original_directory
for folder_name in os.listdir(CLU_directory):
    print(folder_name)
    folder_path = os.path.join(CLU_directory, folder_name)
    if os.path.isdir(folder_path):
        # Find the shapefile in the folder
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.shp'):
                # Read the shapefile into a GeoDataFrame
                gdf = gpd.read_file(os.path.join(folder_path, file_name))
                # Apply negative buffer
                gdf['geometry'] = gdf['geometry'].buffer(-neg_buffer)
                # Calculate the area for each geometry
                gdf['area'] = gdf.geometry.area/4046.85642
                # Filter rows based on acreage
                gdf = gdf[gdf['area'] >= threshold_acreage]
                gdf = gdf.reset_index(names='ID')
                # Calculate the inside point for each geometry
                inside_points = []
                for geom in gdf.geometry:
                    # Check if the geometry is a valid polygon
                    if geom.is_empty:
                        inside_points.append(None)  # Handle non-polygon geometries or empty geometries
                        continue
                    
                    # Calculate the centroid
                    centroid = geom.centroid
                    
                    # Check if the centroid is inside the geometry
                    if geom.contains(centroid):
                        inside_points.append(centroid)
                    else:
                        # Generate a random point inside the geometry
                        random_point = random_point_in_polygon(geom)
                        inside_points.append(random_point)
                
                gdf_points = gdf.copy()
                gdf_points.geometry = inside_points
                gdf_points = gdf_points.dropna()
    
                for year in range(2013, 2024):
                    gdf_points = extract_raster_values(gdf_points, year)
    
                gdf_points = gdf_points[['ID','CDL2013', 'CDL2014', 'CDL2015', 'CDL2016', 
                                         'CDL2017', 'CDL2018', 'CDL2019', 'CDL2020', 
                                         'CDL2021', 'CDL2022', 'CDL2023']].dropna(thresh=4)
                gdf = gdf.merge(gdf_points, on=['ID'], how='inner')
                list_gdfs.append(gdf)
            
# End timing
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

In [ ]:
Uni_CRS = list_gdfs[0].crs
for i in list_gdfs:
    if i.crs != Uni_CRS:
        i.to_crs(Uni_CRS, inplace=True)

In [ ]:
CLU = pd.concat(list_gdfs)

In [ ]:
CLU.head(5)

In [ ]:
CLU.reset_index(inplace=True)
CLU.drop(columns=['ID', 'index'], inplace=True)
CLU.reset_index(inplace=True, names=['ID'])
CLU['ID'] = [f'{state_abb}{str(x).zfill(8)}' for x in CLU['ID']]

In [ ]:
CLU[['ID', 'geometry']].to_file(os.path.join(output_dir, f'CLU_{state_abb}.shp'))
CLU.drop(columns=['geometry']).to_csv(os.path.join(output_dir, f'CLU_{state_abb}.csv'))